Make prediction on unseen data (test.csv) using best_model.pkl file

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import  LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

In [3]:
# Load the unseen dataset from the test.csv file without specifying column names
unseen_data = pd.read_csv('test.csv', header=None)  # no header in the CSV file
unseen_data.head()

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
0,1698001,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.414798,0.000000,1.000000,0.000000,0.202060,...,0.0,1.000000,1262,892,NO,NO,NO,0,0.089686,0.193344
1,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,...,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
2,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,...,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
3,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.870000,4672,3306,YES,NO,YES,0,0.870538,0.405822
4,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,...,0.0,0.870000,4400,3413,YES,NO,YES,2,0.224729,0.870909


In [4]:
unseen_data.tail()

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
1995,1699996,NO,NO,Z6vucL/W0MPoFsgu2ewNXrvNCAQFiKzUJTYuqh6lP28=,yhI9Bw5Q8l1vEll4sw/Tem/jojpE9KwjKvQQIyrAqgU=,1.294118,0.000000,1.000000,0.000000,0.164141,...,0.0,1.000000,1188,918,YES,NO,YES,2,0.198257,0.155724
1996,1699997,NO,NO,LKQ9Uh6tQ3ZrIxAKaPaDEuiYFunnK/2d+oKAfpN9tuY=,h0cPLYjd7nmw9FJsQA+KUsnChH0SajbHjNdfMk47k9o=,1.020217,0.583944,0.625842,1.003516,0.791136,...,0.0,0.720000,4400,3413,YES,NO,YES,0,0.582479,0.778864
1997,1699998,NO,NO,/tuZYGMsFx4A/Ou+jSol6t/TpLRkSl8Ku+1tnQPvwww=,aLEeZ8ZFKt2jQfkG5e9Nmad+QJlfpPmSfQS3CHlL6Ik=,0.354706,0.550882,0.930882,0.207941,0.207500,...,0.0,0.845000,4400,3400,NO,NO,NO,5,0.930588,0.201591
1998,1699999,NO,NO,uMIU2KDOxlgzhYToCFCa3nMxIOPV0WqCnKWfooGaw+8=,4LhhvTzxwvh2SnFtcpaRasyvph66a3YDIQCshAfyS2o=,1.220588,0.102059,0.326176,1.213824,0.942955,...,1.5,0.676667,4400,3400,NO,NO,NO,2,0.256471,0.938182
1999,1700000,NO,NO,WAhQ/X/Jnshuigil3iUfuN/7xgMCklDnEc5N5eLGaVY=,pkNFxQlGfRUOOpf5wTKhFS6Y0QYhdF3f6/IVoWQShY4=,0.699425,0.100695,0.793771,0.183550,0.420569,...,0.0,0.580000,4677,3307,NO,NO,NO,0,0.190807,0.416506


In [5]:
unseen_data.shape

(2000, 146)

In [6]:
# drop irrelevent columns
data = unseen_data.drop(columns=[0],axis=1)
data.head()

,1,2,3,4,5,6,7,8,9,10,...,136,137,138,139,140,141,142,143,144,145
0,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.414798,0.000000,1.000000,0.000000,0.202060,NO,...,0.0,1.000000,1262,892,NO,NO,NO,0,0.089686,0.193344
1,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,...,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
2,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,...,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
3,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,...,0.0,0.870000,4672,3306,YES,NO,YES,0,0.870538,0.405822
4,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,NO,...,0.0,0.870000,4400,3413,YES,NO,YES,2,0.224729,0.870909


In [7]:
print(f"Shape of 'data' : {data.shape}")

Shape of 'data' : (2000, 145)


In [8]:
# check duplicated rows

print(f"No. of duplicated rows : {data.duplicated().sum()}")

No. of duplicated rows : 3


In [9]:
data = data.drop_duplicates()
print(f"No. of duplicated rows : {data.duplicated().sum()}")

No. of duplicated rows : 0


In [10]:
# handelling missing values

print(f"No of missing values : {data.isnull().sum()}")
print(f"Total no of missing values : {data.isnull().sum().sum()}")

No of missing values : 1      291
2      291
3      291
4      291
5        0
      ... 
141      0
142      0
143      0
144      0
145      0
Length: 145, dtype: int64
Total no of missing values : 7404


In [11]:
# droping rows with missing values
data.dropna(inplace=True)
print(f"No of missing values : {data.isnull().sum().sum()}")

No of missing values : 0


In [12]:
# text column name in data
text_column = 3 

# Bag of Words (CountVectorizer)
count_vectorizer = CountVectorizer()
bow_features1 = count_vectorizer.fit_transform(data[text_column])

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_features1 = tfidf_vectorizer.fit_transform(data[text_column])

# Word Embeddings (Word2Vec)
# First, tokenize the text into sentences (list of words)
tokenized_sentences1 = [text.split() for text in data[text_column]]

# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_sentences1, vector_size=100, window=5, min_count=1, sg=1)

# Convert text to word embeddings
word_embeddings1 = []
for sentence in tokenized_sentences1:
    embeddings = [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]
    if embeddings:
        mean_embedding = sum(embeddings) / len(embeddings)
        word_embeddings1.append(mean_embedding)
    else:
        # If no word in the sentence is present in the vocabulary, use zeros as embeddings
        word_embeddings1.append([0] * 100)  # Assuming vector_size=100

word_embeddings1 = pd.DataFrame(word_embeddings1)


In [13]:
word_embeddings1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.001511,-0.004035,-0.004399,-0.004629,-0.005596,-0.005302,-0.008022,0.009519,0.006399,-0.003607,...,0.000147,-0.002285,0.005307,-0.003415,-0.002616,0.008792,-0.009155,-0.003414,-0.002924,0.005669
1,-0.007541,0.000336,-0.003239,-0.001235,0.000770,0.004195,-0.008818,0.009998,-0.004623,0.000677,...,-0.001979,0.000558,-0.004813,0.002784,0.007569,0.006250,0.009572,-0.001049,-0.006044,0.006765
2,-0.004388,0.007569,0.009045,-0.008975,0.009483,-0.009828,-0.006401,0.007836,-0.000308,0.001340,...,0.001765,-0.001598,0.008325,0.003075,-0.009655,-0.008680,0.003849,-0.006123,0.009615,0.003458
3,0.007089,-0.001568,0.007947,-0.009489,-0.008029,-0.006640,-0.004003,0.004989,-0.003814,-0.008320,...,0.007512,0.001498,-0.001265,0.005768,-0.005640,0.000039,0.009457,-0.005481,0.003814,-0.008113
4,0.007592,0.004428,0.004365,-0.005674,0.007436,-0.003959,0.005932,-0.006113,0.006255,0.008800,...,-0.008470,-0.008170,-0.001772,-0.008748,0.005478,0.002712,0.009434,-0.005914,-0.002507,0.006705


In [14]:
# text column name in data
text_column = 4  

# Bag of Words (CountVectorizer)
count_vectorizer = CountVectorizer()
bow_features2 = count_vectorizer.fit_transform(data[text_column])

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_features2 = tfidf_vectorizer.fit_transform(data[text_column])

# Word Embeddings (Word2Vec)
# First, tokenize the text into sentences (list of words)
tokenized_sentences2 = [text.split() for text in data[text_column]]

# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_sentences2, vector_size=100, window=5, min_count=1, sg=1)

# Convert text to word embeddings
word_embeddings2 = []
for sentence in tokenized_sentences2:
    embeddings = [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]
    if embeddings:
        mean_embedding = sum(embeddings) / len(embeddings)
        word_embeddings2.append(mean_embedding)
    else:
        # If no word in the sentence is present in the vocabulary, use zeros as embeddings
        word_embeddings2.append([0] * 100)  # Assuming vector_size=100

word_embeddings2 = pd.DataFrame(word_embeddings2)
# Rename columns starting from 100
word_embeddings2.columns = range(100, 100 + len(word_embeddings2.columns))

In [15]:
word_embeddings2.head()

,100,101,102,103,104,105,106,107,108,109,...,190,191,192,193,194,195,196,197,198,199
0,0.000095,0.003077,-0.006813,-0.001375,0.007669,0.007346,-0.003673,0.002643,-0.008317,0.006205,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
1,-0.009579,0.008943,0.004165,0.009235,0.006644,0.002925,0.009804,-0.004425,-0.006803,0.004227,...,-0.005085,0.001131,0.002883,-0.001536,0.009932,0.008350,0.002416,0.007118,0.005891,-0.005581
2,0.009941,0.002402,0.005128,-0.005669,0.004379,0.000257,-0.009590,0.005068,-0.009009,-0.006469,...,-0.006545,-0.007366,0.002032,-0.009011,0.005789,0.005469,-0.003818,-0.005641,-0.008327,-0.001897
3,0.008132,-0.004457,-0.001068,0.001006,-0.000191,0.001148,0.006114,-0.000020,-0.003246,-0.001511,...,-0.002701,0.000444,-0.003537,-0.000419,-0.000709,0.000823,0.008195,-0.005737,-0.001660,0.005572
4,-0.005156,-0.006668,-0.007777,0.008311,-0.001982,-0.006855,-0.004154,0.005144,-0.002869,-0.003750,...,-0.008977,0.008592,0.004047,0.007470,0.009746,-0.007290,-0.009040,0.005836,0.009391,0.003507


In [16]:
print(f"Shape of 'word_embaddings1 : {word_embeddings1.shape}")
print(f"Shape of 'word_embaddings2 : {word_embeddings2.shape}")

Shape of 'word_embaddings1 : (1465, 100)
Shape of 'word_embaddings2 : (1465, 100)


In [17]:
# concating 'data' and 'word_embaddings' into final_data as dataframe
final_data = pd.concat([data, word_embeddings1,word_embeddings2], axis=1)
final_data.head()

,1,2,3,4,5,6,7,8,9,10,...,190,191,192,193,194,195,196,197,198,199
1,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,...,-0.005085,0.001131,0.002883,-0.001536,0.009932,0.008350,0.002416,0.007118,0.005891,-0.005581
2,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,...,-0.006545,-0.007366,0.002032,-0.009011,0.005789,0.005469,-0.003818,-0.005641,-0.008327,-0.001897
4,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,NO,...,-0.008977,0.008592,0.004047,0.007470,0.009746,-0.007290,-0.009040,0.005836,0.009391,0.003507
5,NO,NO,kM4KU87XvnvKRvf4dN3Tu4zQYq8fpcqhDTFADWdfCg8=,4LhhvTzxwvh2SnFtcpaRasyvph66a3YDIQCshAfyS2o=,1.415919,0.000000,1.000000,0.000000,0.232779,NO,...,0.004774,-0.003262,-0.009268,0.003787,0.007161,-0.005633,-0.007865,-0.002973,-0.004932,-0.002315
6,NO,NO,NMFPnlbm6YWoxpG5KdcfdDMWWSZ7FYAUlJoLtkBBf9k=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.414798,0.000000,1.000000,0.000000,0.323296,NO,...,-0.008977,0.008592,0.004047,0.007470,0.009746,-0.007290,-0.009040,0.005836,0.009391,0.003507


In [18]:
print(f"Shape of 'final_data : {final_data.shape}")

Shape of 'final_data : (1856, 345)


In [19]:
#missing values
print(f"missing values in 'final_data': {final_data.isnull().sum().sum()}")

missing values in 'final_data': 134895


In [20]:
final_data.dropna(inplace=True)
print(f"after handeling, checking missing values in 'final_data': {final_data.isnull().sum().sum()}")
print(f"Shape of 'final_data : {final_data.shape}")

after handeling, checking missing values in 'final_data': 0
Shape of 'final_data : (1074, 345)


In [21]:
final_data.head()

,1,2,3,4,5,6,7,8,9,10,...,190,191,192,193,194,195,196,197,198,199
1,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,...,-0.005085,0.001131,0.002883,-0.001536,0.009932,0.008350,0.002416,0.007118,0.005891,-0.005581
2,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,NO,...,-0.006545,-0.007366,0.002032,-0.009011,0.005789,0.005469,-0.003818,-0.005641,-0.008327,-0.001897
4,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,NO,...,-0.008977,0.008592,0.004047,0.007470,0.009746,-0.007290,-0.009040,0.005836,0.009391,0.003507
5,NO,NO,kM4KU87XvnvKRvf4dN3Tu4zQYq8fpcqhDTFADWdfCg8=,4LhhvTzxwvh2SnFtcpaRasyvph66a3YDIQCshAfyS2o=,1.415919,0.000000,1.000000,0.000000,0.232779,NO,...,0.004774,-0.003262,-0.009268,0.003787,0.007161,-0.005633,-0.007865,-0.002973,-0.004932,-0.002315
6,NO,NO,NMFPnlbm6YWoxpG5KdcfdDMWWSZ7FYAUlJoLtkBBf9k=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.414798,0.000000,1.000000,0.000000,0.323296,NO,...,-0.008977,0.008592,0.004047,0.007470,0.009746,-0.007290,-0.009040,0.005836,0.009391,0.003507


In [22]:
final_data.shape

(1074, 345)

In [23]:
# taking relevent featues only for prediction

'''df = final_data[[32,33,12,6,9,7,29,3,
                       31,28,26,21,30,27,10,
                       24,22,4,16,20]]'''
df = final_data[[59,58,15,21,18,54,53,16,52,7,8,49,19,47,22,51,39,28,50,38]]

In [24]:
print(f"Shape of X :{df.shape}")

Shape of X :(1074, 40)


In [25]:
# Drop duplicate columns
df_filtered = df.loc[:, ~df.columns.duplicated()]

# Print the filtered DataFrame
print("Filtered DataFrame:")
print(df_filtered)

# Print the remaining columns
print("\nRemaining columns:")
print(df_filtered.columns)

# Print the shape of the filtered DataFrame
print("\nShape of filtered DataFrame:", df_filtered.shape)

Filtered DataFrame:
            59   58    15    21    18      54      53        16    52  \
1     0.838115  3.0   3.0  0.88   7.0  3311.0  4672.0  0.895500  0.84   
2     0.526906  7.0   4.0  1.00  10.0   892.0  1263.0  0.536996  1.00   
4     0.228245  7.0   5.0  0.73  10.0  3413.0  4400.0  0.238793  0.90   
5     0.496637  5.0   2.0  1.00   8.0   892.0  1263.0  0.516816  1.00   
6     0.819507  2.0   1.0  1.00   4.0   892.0  1262.0  0.828475  1.00   
...        ...  ...   ...   ...   ...     ...     ...       ...   ...   
1460  0.240399  0.0   5.0  0.88   7.0  3307.0  4677.0  0.180829  0.86   
1461  0.592552  6.0  31.0  1.00  10.0   913.0  1186.0  0.605696  1.00   
1462  0.251588  2.0   9.0  0.91   7.0  3307.0  4677.0  0.801935  0.88   
1463  0.645766  0.0   1.0  0.90   5.0  3413.0  4400.0  0.644887  0.90   
1464  0.556396  0.0   1.0  0.89   4.0  3307.0  4677.0  0.649834  0.86   

            7         8     49        19        47      22   51        39  \
1     0.945938  0.317427  

In [26]:
print(f"Shape of X :{df_filtered.shape}")

Shape of X :(1074, 20)


## Feature selection

In [27]:
# Encode categorical features (if any)
categorical_columns = df_filtered.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()
df_filtered[categorical_columns] = df_filtered[categorical_columns].apply(label_encoder.fit_transform)

# Now, check if there are any remaining non-numeric columns
non_numeric_columns = df_filtered.select_dtypes(exclude=['number']).columns
print("Non-numeric columns:", non_numeric_columns)

Non-numeric columns: Index([], dtype='int64')


## Prediction (on unsee data)

In [28]:
# Load the trained classifier model
best_clf_model = joblib.load("best__model.pkl")

# 4. Make predictions on the preprocessed test data using the loaded model
predictions = best_clf_model.predict(df_filtered)

c:\0_ProjectTask\env\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [29]:
print(predictions)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [30]:
import pandas as pd

# Convert the NumPy array to a DataFrame with appropriate column names
predictions_df = pd.DataFrame(predictions, columns=[f'predicted_label_{i}' for i in range(1, 34)])

# Save the DataFrame to a CSV file
predictions_df.to_csv('Predictions.csv', index=False)


In [31]:
# since predictions_df contains the DataFrame with predicted labels

# Generate the 'id_label' column
id_labels = [f'{i}_y{j}' for i in range(1, len(predictions_df) + 1) for j in range(1, 34)]

# Flatten the predictions array
predicted_labels = predictions_df.values.flatten()

# Create a DataFrame with 'id_label' and 'pred' columns
result_df = pd.DataFrame({'id_label': id_labels, 'pred': predicted_labels})

# Save the DataFrame to a CSV file
result_df.to_csv('Predictions.csv', index=False)


In [34]:
pred = pd.read_csv("Predictions.csv")
pred.head()

,id_label,pred
0,1_y1,0.0
1,1_y2,0.0
2,1_y3,0.0
3,1_y4,0.0
4,1_y5,0.0


In [35]:
pred.shape

(35442, 2)